In [1]:
%cd ..
import os
import time

/data/apperception/apperception


In [2]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
# disable_cache()

/home/local/apperception/.installs/mambaforge/envs/apperception/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Using cache found in /data/apperception/apperception/weights/ultralytics_yolov5_master


Using cuda:0


YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
NUSCENES_RAW_DATA = "NUSCENES_RAW_DATA"
if NUSCENES_RAW_DATA in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA]
else:
    RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini"

In [4]:
import cv2
def construct_video(frames, size=(1600, 900), base=False, vid_name=None):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/fig13_"+vid_name if vid_name else f"./outputs/{'fig13_base' if base else 'fig13_optimized'}.avi"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [5]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
obj2 = objects[1]
cam = camera

world = world.filter(
    (obj1.id != obj2.id) &
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck')) &
    (F.like(obj2.type, 'car') | F.like(obj2.type, 'truck')) &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
    (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2.0) &
    (F.distance(cam.ego, obj2.trans@cam.time) < 50) &
    (F.view_angle(obj2.trans@cam.time, cam.ego) < 70 / 2.0) &
    F.contains_all('intersection', [obj1.trans, obj2.trans]@cam.time) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 50, 180) &
    F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.ego), -135, -50) &
    (F.min_distance(cam.ego, 'intersection') < 10) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100)
)


In [6]:
# preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT'])

In [7]:
# base_query_start = time.time()
# id_time_camId_filename_base = world.get_id_time_camId_filename(2)
# print(f'based query time: {time.time()-base_query_start}')

In [8]:
# id_time_camId_filename_base

In [9]:
# base_results = [e[4] for e in id_time_camId_filename_base]
# construct_video(base_results, base=True)

In [10]:
preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT'], base=False)

YOLOv5 🚀 2022-11-23 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Using cache found in /data/apperception/weights/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


scene-0757-CAM_FRONT --------------------------------------------------------------------------------
Stage:  InView
237
None
  filtered frames: 97.46835443037975%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  DecodeFrame.ParallelDecodeFrame
None
237
  filtered frames: 97.46835443037975%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.YoloDetection
None
237
  filtered frames: 97.46835443037975%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

237it [00:00, 3202.76it/s]


None
237
  filtered frames: 97.46835443037975%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  DetectionEstimation
ego_speed:  1.3581198756421884


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 236/236 [00:17<00:00, 13.76it/s]


number of skipped 7
{None: 78, 'ego_exit_segment': 3, 'car_exit_segment': 22, 'meet_up': 127}
total_run_time 17.157092571258545
total_detection_time 12.889576196670532
total_generate_sample_plan_time 4.12225866317749
237
237
  filtered frames: 97.0464135021097%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK.
Stage:  Tracking2D.StrongSORT
None
237
  filtered frames: 97.0464135021097%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK.
Stage:  Tracking3D.From2DAndRoad


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 237/237 [00:00<00:00, 7625.42it/s]


None
237
  filtered frames: 97.0464135021097%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK.
Stage:  SegmentTrajectory.FromTracking3D
None
237
  filtered frames: 97.0464135021097%
......KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK.
Time taken to insert trajectories: 0.1581733226776123
info found []
total preprocess time 25.348026752471924


In [11]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(2)
print(f'optimized query time: {time.time()-optimized_query_start}')

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId,t1.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        JOIN (SELECT * FROM item_general_trajectory) as t1 USING (cameraId)
        JOIN Cameras USING (cameraId)
        WHERE ((t0.itemId<>t1.itemId) AND (t0.objectType LIKE 'car' OR t0.objectType LIKE 'truck') AND (t1.objectType LIKE 'car' OR t1.objectType LIKE 'truck') AND angleBetween(facingRelative(egoHeading,roadDirection(egoTranslation,egoHeading)),-15,15) AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t0.translations,timestamp),egoHeading,egoTranslation)<35.0) AND (ST_Distance(egoTranslation,valueAtTimestamp(t1.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t1.translations,timestamp),egoHeading,egoTranslation)<35.0) AND (EXISTS(SELECT 1
            FROM SegmentPolygon
     

In [12]:
optimized_results = [e[4] for e in id_time_camId_filename]
construct_video(optimized_results)

In [14]:
# false_negative = [f for f in base_results if f not in optimized_results]
# print(len(false_negative))
# construct_video(false_negative, vid_name="false_negative.avi")

In [15]:
# false_positive = [f for f in optimized_results if f not in base_results]
# print(len(false_positive))
# construct_video(false_positive, vid_name="false_positive.avi")

In [ ]:
# [e for e in id_time_camId_filename if e[4] in false_positive]

In [ ]:
investigation = [[116, 316.0, 455.0, 296.0, 95.0], [161, 316.0, 454.0, 302.0, 94.0]]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

im = Image.open('/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini/sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125362404.jpg')

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(im)

# Create a Rectangle patch
for obj in investigation:
    obj_id = obj[0]
    x,y,w,h = obj[1:5]
    plt.text(x, y+15, 'obj'+str(obj_id))
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

plt.show()